In [ ]:
import numpy as np
from plotly import graph_objs as go
from plotly.subplots import make_subplots
import plotly.express as px
from time import sleep
import datetime
from scipy.optimize import curve_fit
import pandas as pd
from tqdm.notebook import trange
import os
from joblib import Parallel, delayed
import time
import multiprocessing
from scipy.interpolate import interp1d
import csv
from collections import defaultdict
import plotly.io as pio

In [ ]:
# To install orca and related packages
%%capture
!pip install plotly>=4.7.1
!wget https://github.com/plotly/orca/releases/download/v1.2.1/orca-1.2.1-x86_64.AppImage -O /usr/local/bin/orca
!chmod +x /usr/local/bin/orca
!apt-get install xvfb libgtk2.0-0 libgconf-2-4

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Check the file path
!ls "/content/gdrive/My Drive/Research/ML_PROJECTS_ON_SUPERCONDUCTING_QUBITS/Logspace/18Dec2023"

 Batch_Timings_18Dec2023  'T1 and T2.html'    T1_T2_data.npz
 job_retrieve.ipynb	  'T1 and T2.ipynb'   t1_t2_exp.ipynb


In [ ]:
# Create the the main file path
path = F"/content/gdrive/My Drive/Research/ML_PROJECTS_ON_SUPERCONDUCTING_QUBITS/Logspace/18Dec2023"

### Figure template

In [ ]:
# Figure template
fig_template = go.layout.Template()
fig_template.layout = {
    'template': 'simple_white+presentation',
    'autosize': False,
    'width': 800,
    'height': 450,
    # 'opacity': 0.2,
    'xaxis': {
        'title': 'X axis',
        'ticks': 'inside',
        'mirror': 'ticks',
        'linewidth': 2,
        'tickwidth': 2,
        'ticklen': 6,
        'showline': True,
        'showgrid': False,
        'zerolinecolor': 'white',
        },
    'yaxis': {
        'title': 'Y axis',
        'ticks': 'inside',
        'mirror': 'ticks',
        'linewidth': 2,
        'tickwidth': 2,
        'ticklen': 6,
        'showline': True,
        'showgrid': False,
        'zerolinecolor': 'white'
        },
    'font':{'family':'mathjax',
            'size': 18,
            }
}

### Load experimental and traning data

In [ ]:
data_train1 = np.load(path+"Data analysis/Lor_1.npz")

In [ ]:
c_dataf = data_train1['arr_0']      # C data
T_in = data_train1['arr_1']         # Time vector for data generation
s_dataf = data_train1['arr_2']      # S data
w0 = data_train1['arr_3']           # Omega vector for data generation
T_train = data_train1['arr_4']      # Time vector for training data (based on the experimental data)
w_train = data_train1['arr_5']      # Omega vector for training data
T2_span = data_train1['arr_6']      # T2 distribution

In [ ]:
# For data interpolation
def interpData(x,y,xNew):
    f_interp = interp1d(x,y)
    yNew = f_interp(xNew)
    return yNew

# For preparing training data: Add random noise, then replace low values with zeros
# Run this cell multiple times to generate sets with different random noise but same underlying curves
def prepare_trainData(c_in,T_in,T_train,noiseMax=0.03,cutOff=0.03):
  c_train = interpData(T_in,c_in,T_train)
  for i in trange(c_in.shape[0]):
    c_train[i,:] = c_train[i,:] + np.random.normal(0,noiseMax*2/3,size=c_train.shape[1])
    cut = np.squeeze(np.argwhere(c_train[i,:]<=cutOff+np.random.normal(0,noiseMax*2/3,1)))
    if cut.size > 1:
      c_train[i,cut[0]-1:] = 0
    elif cut.size == 1:
      c_train[i,cut-1:] = 0
  return c_train

In [ ]:
c_dataf.shape

(5222, 375)

In [ ]:
# Interpolate, add random noise to experimental conditions, cut the tail of the data
c_train = prepare_trainData(c_dataf,T_in,T_train)

  0%|          | 0/5222 [00:00<?, ?it/s]

In [ ]:
# Intepolate
s_train = interpData(w0,s_dataf,w_train)

In [ ]:
# data_24sept = np.load(path+'T1_T2_25sept/24thsept23/T1_T2_data.npz')
data_24sept = np.load(path+'T1_T2_14Oct_400us/T1_T2_data.npz')
data_24sept.files

['pop_1', 'pop_0', 'xval', 'counts_t1', 'counts_t2']

In [ ]:
data = np.load(path+'/T1_T2_data.npz')

In [ ]:
data.files

['pop_1', 'pop_0', 'xval', 'counts_t1', 'counts_t2']

In [ ]:
data_24sept['xval'].shape

(150,)

In [ ]:
data_24sept['pop_1'].shape

(10, 150, 7)

In [ ]:
data_24sept['pop_0'].shape

(10, 150, 7)

In [ ]:
repeats = np.linspace(0,9,10)

In [ ]:
# %%
# Stretched-exponential function
def stretchExp(T0,T2,p,A):
    C = A*np.exp(-((T0/T2)**p))
    return C

# %%
# Stretched-exponential function
def simpleExp(T0,T1):
    C = np.exp(-(T0/T1))
    return C


# %%
# Extract T2, p, and amplitude by fitting the data with stretched-exponential function
def fit_stretchExp(C,T0):
    params = curve_fit(stretchExp, T0, C, bounds=([30e-6,1,0.97],[400e-6,2,1.03]))
    T2, p, A = params[0]
    T2err, perr, Aerr = np.sqrt(np.diag(params[1]))
    return T2, p, A, T2err, perr, Aerr

# %%
# Extract T1 by fitting the data with simple-exponential function
def fit_simpleExp(signal,T0):
    params = curve_fit(simpleExp, T0, signal, bounds=([10e-6],[1000e-6]))
    T1 = params[0]
    T1err = np.sqrt(np.diag(params[1]))
    return T1, T1err


# %%
# Normalise experimental decoherence data from 1 to 0
def normalise(data):
  c_data = (data-0.5)/0.5
  return c_data


# %%
# For processing the experimental data training data: interpolation, and random cut-off
def prepare_expData(c_in,T_in,T_train,cutOff=0.03):
  if np.max(T_train) < np.max(T_in):
    c_predict = interpData(T_in,c_in,T_train)
    for i in trange(c_in.shape[0]):
      cut = np.squeeze(np.argwhere(c_predict[i,:]<=cutOff))
      if cut.size > 1:
        c_predict[i,cut[0]-1:] = 0
      elif cut.size == 1:
        c_predict[i,cut-1:] = 0
  else:
    T_train1 = np.squeeze(T_train[np.argwhere(T_train<=np.max(T_in))])
    zero_size = T_train.size - T_train1.size
    c_predict = interpData(T_in,c_in,T_train1)
    for i in trange(c_in.shape[0]):
      cut = np.squeeze(np.argwhere(c_predict[i,:]<=cutOff))
      if cut.size > 1:
        c_predict[i,cut[0]-1:] = 0
      elif cut.size == 1:
        c_predict[i,cut-1:] = 0
    c_predict = np.concatenate((c_predict,np.zeros((c_in.shape[0],zero_size))),axis=1)
  return c_predict

### Plot data

#### Hahn-echo T<sub>2</sub>

In [ ]:
exp_select = data['pop_0'][:,:,(0,3)]
exp_select = normalise(np.concatenate((exp_select[:,:,0],exp_select[:,:,1])))
# c_predict= prepare_expData(exp_select,data['xval'],T_train)
# np.savez_compressed(path+F"Data analysis/24sept_q0q3",ml_input = c_predict)

In [ ]:
data_input = np.load(path+"Data analysis/24sept_q0q3.npz")
c_predict = data_input['ml_input']

In [ ]:
exp_select.shape

(20, 150)

In [ ]:
fig=go.Figure()
for i in range(exp_select.shape[0]//2):
  fig.add_scatter(x=data['xval']*1e6,y=exp_select[i+10,:],name=f'Repeat {i}',
                  mode="lines+markers",opacity=0.5,line=dict(color=px.colors.qualitative.Vivid[i]))
  # fig.add_scatter(x=T_train*1e6,y=c_predict[i,:],name=f'ML-Input {i}',
  #                 mode="lines",opacity=1,line=dict(width=3,color=px.colors.qualitative.Vivid[i]))

fig.update_layout(template=fig_template,  title = 'Qubit 0',
                  width = 1000,
                  xaxis = dict(title='Evolution time (\N{greek small letter mu}s)',range=[0,350]),
                  yaxis = dict(title='Coherence'),
                 )

In [ ]:
fig=go.Figure()
for i in range(exp_select.shape[0]//2):
  fig.add_scatter(x=data['xval']*1e6,y=exp_select[i+10,:],name=f'Repeat {i}',
                  mode="lines+markers",opacity=0.5,line=dict(color=px.colors.qualitative.Vivid[i]))
  fig.add_scatter(x=T_train*1e6,y=c_predict[i+10,:],name=f'ML-Input {i}',
                  mode="lines",opacity=1,line=dict(width=3,color=px.colors.qualitative.Vivid[i]))

fig.update_layout(template=fig_template,  title = 'Qubit 3',
                  width = 1000,
                  xaxis = dict(title='Evolution time (\N{greek small letter mu}s)',range=[0,350]),
                  yaxis = dict(title='Coherence'),
                 )

##### Fitted

In [ ]:
# Fitting

param = np.zeros((3,(data['pop_0'].shape[2])))
fit_curves = np.zeros((data['pop_0'].shape[1],data['pop_0'].shape[2]))
for i in range(data['pop_0'].shape[2]):
  param[0,i],param[1,i],param[2,i],_,_,_ = fit_stretchExp(normalise(np.squeeze(data['pop_0'][0,:,i])),data['xval'])
  fit_curves[:,i] = stretchExp(data['xval'],param[0,i],param[1,i],param[2,i])

In [ ]:
fig=go.Figure()
for i in range(data['pop_0'].shape[2]):
  fig.add_scatter(x=data['xval']*1e6,y=normalise(data['pop_0'][0,:,i]),name=f'Qubit {i}',
                  mode="lines+markers",opacity=0.5,line=dict(color=px.colors.qualitative.Vivid[i]))
  fig.add_scatter(x=data['xval']*1e6,y=fit_curves[:,i],name='T<sub>2</sub> = '+f'{int(param[0,i]*1e6)}'+' \N{greek small letter mu}s',
                  mode="lines",line=dict(width=3,color=px.colors.qualitative.Vivid[i]),opacity=1)

fig.update_layout(template=fig_template, width = 1000,
                  xaxis = dict(title='Evolution time (\N{greek small letter mu}s)',range=[0,400]),
                  yaxis = dict(title='Coherence'),
                 )

In [ ]:
pio.write_image(fig,path+'T1_T2_25sept/24thsept23/T2_t0',format='pdf')

In [ ]:
fig.write_html(path+'T1_T2_25sept/24thsept23/T2_t0.html')

#### T<sub>1</sub>

In [ ]:
t1_data = np.squeeze(data['pop_1'][0,:,:])
t1_data = t1_data/t1_data[0,:]

In [ ]:
# Fitting

param_t1 = np.zeros((data['pop_1'].shape[2]))
fit_curves_t1 = np.zeros((data['pop_1'].shape[1],data['pop_1'].shape[2]))
for i in range(data['pop_1'].shape[2]):
  param_t1[i],_ = fit_simpleExp(t1_data[:,i],data['xval'])
  fit_curves_t1[:,i] = simpleExp(data['xval'],param_t1[i])

In [ ]:
fig=go.Figure()
for i in range(data['pop_1'].shape[2]):
  fig.add_scatter(x=data['xval']*1e6,y=t1_data[:,i],name=f'Qubit {i}',
                  mode="lines+markers",opacity=0.5,line=dict(color=px.colors.qualitative.Vivid[i]))
  fig.add_scatter(x=data['xval']*1e6,y=fit_curves_t1[:,i],name='T<sub>1</sub> = '+f'{int(param_t1[i]*1e6)}'+' \N{greek small letter mu}s',
                  mode="lines",line=dict(width=3,color=px.colors.qualitative.Vivid[i]),opacity=1)

fig.update_layout(template=fig_template, width = 1000,
                  xaxis = dict(title='Evolution time (\N{greek small letter mu}s)',range=[0,300]),
                  yaxis = dict(title='Coherence'),
                 )

In [ ]:
fig.write_html(path+'T1_T2_25sept/24thsept23/T1_t0.html')

#### Heatmaps

In [ ]:
fig = make_subplots(rows=3, cols=4, subplot_titles=('Qubit 0', 'Qubit 1', 'Qubit 2','Qubit 3', 'Qubit 4', 'Qubit 5','Qubit 6'))
fig.add_heatmap(x=data['xval']*1e6,y=repeats,z=np.squeeze(data['pop_0'][:,:,0]), row=1, col=1, zmin=0.5, zmax=1, showscale=True,
                colorbar=dict(len=0.5,thickness=20,y=0.65),
                )
fig.add_heatmap(x=data['xval']*1e6,y=repeats,z=np.squeeze(data['pop_0'][:,:,1]), row=1, col=2, zmin=0.5, zmax=1, showscale=False,)
fig.add_heatmap(x=data['xval']*1e6,y=repeats,z=np.squeeze(data['pop_0'][:,:,2]), row=1, col=3, zmin=0.5, zmax=1, showscale=False,)
fig.add_heatmap(x=data['xval']*1e6,y=repeats,z=np.squeeze(data['pop_0'][:,:,3]), row=1, col=4, zmin=0.5, zmax=1, showscale=False,)
fig.add_heatmap(x=data['xval']*1e6,y=repeats,z=np.squeeze(data['pop_0'][:,:,4]), row=2, col=1, zmin=0.5, zmax=1, showscale=False,)
fig.add_heatmap(x=data['xval']*1e6,y=repeats,z=np.squeeze(data['pop_0'][:,:,5]), row=2, col=2, zmin=0.5, zmax=1, showscale=False,)
fig.add_heatmap(x=data['xval']*1e6,y=repeats,z=np.squeeze(data['pop_0'][:,:,6]), row=2, col=3, zmin=0.5, zmax=1, showscale=False,)
fig.update_layout(template=fig_template,
                  width = 1200,
                  height = 1000,
                  xaxis1 = dict(title='',range=[0,300]),
                  yaxis1 = dict(title='Repeats'),
                  xaxis2 = dict(title='',range=[0,300]),
                  yaxis2 = dict(title=''),
                  xaxis3 = dict(title='',range=[0,300]),
                  yaxis3 = dict(title=''),
                  xaxis4 = dict(title='Evolution time (\N{greek small letter mu}s)',range=[0,300]),
                  yaxis4 = dict(title=''),
                  xaxis5 = dict(title='Evolution time (\N{greek small letter mu}s)',range=[0,300]),
                  yaxis5 = dict(title='Repeats'),
                  xaxis6 = dict(title='Evolution time (\N{greek small letter mu}s)',range=[0,300]),
                  yaxis6 = dict(title=''),
                  xaxis7 = dict(title='Evolution time (\N{greek small letter mu}s)',range=[0,300]),
                  yaxis7 = dict(title=''),
                 )


fig

In [ ]:
fig = make_subplots(rows=3, cols=4, subplot_titles=('Qubit 0', 'Qubit 1', 'Qubit 2','Qubit 3', 'Qubit 4', 'Qubit 5','Qubit 6'))
fig.add_heatmap(x=data_24sept['xval']*1e6,y=repeats,z=np.squeeze(data_24sept['pop_1'][:,:,0]), row=1, col=1, zmin=0, zmax=1, showscale=True,
                colorbar=dict(len=0.5,thickness=20,y=0.65),
                )
fig.add_heatmap(x=data_24sept['xval']*1e6,y=repeats,z=np.squeeze(data_24sept['pop_1'][:,:,1]), row=1, col=2, zmin=0, zmax=1, showscale=False,)
fig.add_heatmap(x=data_24sept['xval']*1e6,y=repeats,z=np.squeeze(data_24sept['pop_1'][:,:,2]), row=1, col=3, zmin=0, zmax=1, showscale=False,)
fig.add_heatmap(x=data_24sept['xval']*1e6,y=repeats,z=np.squeeze(data_24sept['pop_1'][:,:,3]), row=1, col=4, zmin=0, zmax=1, showscale=False,)
fig.add_heatmap(x=data_24sept['xval']*1e6,y=repeats,z=np.squeeze(data_24sept['pop_1'][:,:,4]), row=2, col=1, zmin=0, zmax=1, showscale=False,)
fig.add_heatmap(x=data_24sept['xval']*1e6,y=repeats,z=np.squeeze(data_24sept['pop_1'][:,:,5]), row=2, col=2, zmin=0, zmax=1, showscale=False,)
fig.add_heatmap(x=data_24sept['xval']*1e6,y=repeats,z=np.squeeze(data_24sept['pop_1'][:,:,6]), row=2, col=3, zmin=0, zmax=1, showscale=False,)
fig.update_layout(template=fig_template,
                  width = 1200,
                  height = 1000,
                  xaxis1 = dict(title='',range=[0,300]),
                  yaxis1 = dict(title='Repeats'),
                  xaxis2 = dict(title='',range=[0,300]),
                  yaxis2 = dict(title=''),
                  xaxis3 = dict(title='',range=[0,300]),
                  yaxis3 = dict(title=''),
                  xaxis4 = dict(title='Evolution time (\N{greek small letter mu}s)',range=[0,300]),
                  yaxis4 = dict(title=''),
                  xaxis5 = dict(title='Evolution time (\N{greek small letter mu}s)',range=[0,300]),
                  yaxis5 = dict(title='Repeats'),
                  xaxis6 = dict(title='Evolution time (\N{greek small letter mu}s)',range=[0,300]),
                  yaxis6 = dict(title=''),
                  xaxis7 = dict(title='Evolution time (\N{greek small letter mu}s)',range=[0,300]),
                  yaxis7 = dict(title=''),
                 )


fig

In [ ]:
pio.write_image(fig,path+'/plots/endor_final',format='pdf')

### Checking that the noise spectra generates the corresponding decoherence curve

In [ ]:
# %%
# Create CPMG-like pulse timing array
def cpmgFilter(n, Tmax):
    tpi = np.empty([n])
    for i in range(n):
        tpi[i]= Tmax*(((i+1)-0.5)/n)
    return tpi


# %%
# Generate filter function for a given pulse sequence
def getFilter(n,w0,piLength,Tmax):
    tpi = cpmgFilter(n,Tmax)
    f = 0
    for i in range(n):
        f = ((-1)**(i+1))*(np.exp(1j*w0*tpi[i]))*np.cos((w0*piLength)/2) + f

    fFunc = (1/2)*((np.abs(1+((-1)**(n+1))*np.exp(1j*w0*Tmax)+2*f))**2)/(w0**2)
    return fFunc


# %%
# Generate decoherence curve corresponding to a noise spectrum (input shape = variable1.size x w.size)
def getCoherence(S,w0,T0,n,piLength):
    steps = T0.size
    C_invert = np.empty([S.shape[0],steps,])
    for i in trange(steps):
        integ = getFilter(n,np.squeeze(w0),piLength,T0[i])*S/np.pi
        integ_ans = np.trapz(y=integ,x=np.squeeze(w0))
        C_invert[:,i] = np.exp(integ_ans)
    return C_invert

In [ ]:
T_train

array([1.00000000e-06, 3.39041096e-06, 5.78082192e-06, 8.17123288e-06,
       1.05616438e-05, 1.29520548e-05, 1.53424658e-05, 1.77328767e-05,
       2.01232877e-05, 2.25136986e-05, 2.49041096e-05, 2.72945205e-05,
       2.96849315e-05, 3.20753425e-05, 3.44657534e-05, 3.68561644e-05,
       3.92465753e-05, 4.16369863e-05, 4.40273973e-05, 4.64178082e-05,
       4.88082192e-05, 5.11986301e-05, 5.35890411e-05, 5.59794521e-05,
       5.83698630e-05, 6.07602740e-05, 6.31506849e-05, 6.55410959e-05,
       6.79315068e-05, 7.03219178e-05, 7.27123288e-05, 7.51027397e-05,
       7.74931507e-05, 7.98835616e-05, 8.22739726e-05, 8.46643836e-05,
       8.70547945e-05, 8.94452055e-05, 9.18356164e-05, 9.42260274e-05,
       9.66164384e-05, 9.90068493e-05, 1.01397260e-04, 1.03787671e-04,
       1.06178082e-04, 1.08568493e-04, 1.10958904e-04, 1.13349315e-04,
       1.15739726e-04, 1.18130137e-04, 1.20520548e-04, 1.22910959e-04,
       1.25301370e-04, 1.27691781e-04, 1.30082192e-04, 1.32472603e-04,
      

In [ ]:
n_select = [0,1000,2000,3000,4000,5000]
T_select = np.linspace(1e-6,350e-6,350)
c_check = getCoherence(s_train[n_select,:],w_train,T_select,1,100e-9)

  0%|          | 0/350 [00:00<?, ?it/s]

In [ ]:
c_check.shape

(6, 350)

In [ ]:
fig = go.Figure()
for i in range(c_check.shape[0]):
  fig.add_scatter(x=T_train*1e6,y=c_train[n_select[i],:],name=f'Actual {i}',
                  mode="markers",opacity=0.5,line=dict(color=px.colors.qualitative.Vivid[i]))
  fig.add_scatter(x=T_select*1e6,y=c_check[i,:],name=f'Predicted {i}',
                  mode="lines",opacity=1,line=dict(width=3,color=px.colors.qualitative.Vivid[i]))

fig.update_layout(template=fig_template,
                  width = 1000,
                  xaxis = dict(title='Evolution time (\N{greek small letter mu}s)',range=[0,350]),
                  yaxis = dict(title='Coherence'),
                 )